In [1]:
import xarray as xr
import matplotlib.pyplot as plt
from stl import mesh
from scipy import spatial
import numpy as np
from tqdm.auto import tqdm

In [14]:
antdat = xr.open_dataset('control_new.nc')['h'][-1, :, :]
lgm_run = xr.open_dataset('fort.92.nc')['h'][128]

In [3]:
#x_list: list of x coordinates of your grid
#y_list: list of y coordinates of your grid
#data: 2-d gridded data on grid points defined by x_list; should have shape (len(y_list), len(x_list))
#filename: output name 
#assumes x_list and y_list are increasing
#base_depth: value to set base layer datapoints at
#z_scale: what to scale data values by when plotting
#vertex_scale: what to scale all vertices by when outputting
def grid_to_stl(x_list, y_list, data, filename, base_depth = -50, z_scale = 1/7, vertex_scale = 1/30):
    if filename[-4:] != '.stl':
        filename = filename + '.stl'
    
    data_with_guard = np.zeros(tuple(s+2 for s in data.shape), data.dtype)
    data_with_guard[tuple(slice(1,-1) for s in data.shape)] = data

    x_list_with_guard = np.concatenate([[x_list[0] * 0.9999], x_list, [x_list[-1] * 1.0001]])
    y_list_with_guard = np.concatenate([[y_list[0] * 0.9999], y_list, [y_list[-1] * 1.0001]])

    nwidth = len(x_list_with_guard)
    nlength = len(y_list_with_guard)

    vertices = np.zeros([nwidth*nlength*2, 3])
    faces = []
    for i, x in tqdm(enumerate(x_list_with_guard)):
        for j, y in enumerate(y_list_with_guard):
            vertices[i + nwidth*j] = [x, y, base_depth]
            vertices[i + nwidth*j + nwidth*nlength] = [x, y, data_with_guard[j, i] * z_scale]
            index = i + nwidth*j
            
            # if i == 0:
            #     left_edge.append(index)
            #     left_edge.append(index + nwidth*nlength)
            # if i == nwidth - 1:
            #     right_edge.append(index)
            #     right_edge.append(index + nwidth*nlength)
            # if j == 0:
            #     bottom_edge.append(index)
            #     bottom_edge.append(index + nwidth*nlength)
            # if j == nlength - 1:
            #     top_edge.append(index)
            #     top_edge.append(index + nwidth*nlength)
            
            if (i+j) % 2 == 0:
                faces.append([index, index+1, index+nwidth]) if i!= nwidth-1 and j!=nlength-1 else 1
                faces.append([index + nwidth*nlength, index + 1 + nwidth*nlength, 
                              index + nwidth + nwidth*nlength]) if i!= nwidth-1 and j!=nlength-1 else 1
                
                faces.append([index, index-1, index+nwidth]) if i!= 0 and j!=nlength-1 else 1
                faces.append([index + nwidth*nlength, index - 1 + nwidth*nlength, 
                              index + nwidth + nwidth*nlength]) if i!= 0 and j!=nlength-1 else 1
        
                faces.append([index, index+1, index-nwidth]) if i!= nwidth-1 and j!=0 else 1
                faces.append([index + nwidth*nlength, index + 1 + nwidth*nlength, 
                              index - nwidth + nwidth*nlength]) if i!= nwidth-1 and j!=0 else 1
        
                faces.append([index, index-1, index-nwidth]) if i!= 0 and j!=0 else 1
                faces.append([index + nwidth*nlength, index - 1 + nwidth*nlength, 
                              index - nwidth + nwidth*nlength]) if i!= 0 and j!=0 else 1
    
    vertices *= vertex_scale
    
    faces.append([0, nwidth-1, nwidth*nlength])
    faces.append([nwidth-1, nwidth*nlength, nwidth*nlength+nwidth-1])
    
    faces.append([0, nwidth*(nlength-1), nwidth*nlength])
    faces.append([(nlength-1)*nwidth, nwidth*nlength, (nlength-1)*nwidth+nwidth*nlength])
    
    faces.append([nwidth-1, nwidth-1+(nlength-1)*nwidth, nwidth-1+nwidth*nlength])
    faces.append([nwidth-1+(nlength-1)*nwidth, nwidth-1+nwidth*nlength, nwidth-1+(nlength-1)*nwidth+nwidth*nlength])
    
    faces.append([(nlength-1)*nwidth, nwidth-1+(nlength-1)*nwidth, (nlength-1)*nwidth+nwidth*nlength])
    faces.append([nwidth-1+(nlength-1)*nwidth, (nlength-1)*nwidth+nwidth*nlength, nwidth-1+(nlength-1)*nwidth+nwidth*nlength])
    faces = np.asarray(faces)

    myramid_mesh = mesh.Mesh(
      np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype)
    )
    for i, f in enumerate(faces):
      for j in range(3):
          myramid_mesh.vectors[i][j] = vertices[f[j],:]

    myramid_mesh.save(filename)

In [18]:
grid_to_stl(lgm_run['x1'].data, lgm_run['y1'].data, lgm_run.data, 'antarctica_lgm')

0it [00:00, ?it/s]

In [5]:
nwidth = len(antdat['x1'].data) #i
nlength = len(antdat['y1'].data) #j
print(nwidth, nlength)

vertices = np.zeros([nwidth*nlength*2, 3])
left_edge = []
right_edge = []
top_edge = []
bottom_edge = []
faces = []
for i, x in tqdm(enumerate(antdat['x1'].data)):
    for j, y in enumerate(antdat['y1'].data):
        vertices[i + nwidth*j] = [x, y, -50]
        vertices[i + nwidth*j + nwidth*nlength] = [x, y, antdat[j, i]/7]
        index = i + nwidth*j
        
        if i == 0:
            left_edge.append(index)
            left_edge.append(index + nwidth*nlength)
        if i == nwidth - 1:
            right_edge.append(index)
            right_edge.append(index + nwidth*nlength)
        if j == 0:
            bottom_edge.append(index)
            bottom_edge.append(index + nwidth*nlength)
        if j == nlength - 1:
            top_edge.append(index)
            top_edge.append(index + nwidth*nlength)
        
        if (i+j) % 2 == 0:
            faces.append([index, index+1, index+nwidth]) if i!= nwidth-1 and j!=nlength-1 else 1
            faces.append([index + nwidth*nlength, index + 1 + nwidth*nlength, 
                          index + nwidth + nwidth*nlength]) if i!= nwidth-1 and j!=nlength-1 else 1
            
            faces.append([index, index-1, index+nwidth]) if i!= 0 and j!=nlength-1 else 1
            faces.append([index + nwidth*nlength, index - 1 + nwidth*nlength, 
                          index + nwidth + nwidth*nlength]) if i!= 0 and j!=nlength-1 else 1
    
            faces.append([index, index+1, index-nwidth]) if i!= nwidth-1 and j!=0 else 1
            faces.append([index + nwidth*nlength, index + 1 + nwidth*nlength, 
                          index - nwidth + nwidth*nlength]) if i!= nwidth-1 and j!=0 else 1
    
            faces.append([index, index-1, index-nwidth]) if i!= 0 and j!=0 else 1
            faces.append([index + nwidth*nlength, index - 1 + nwidth*nlength, 
                          index - nwidth + nwidth*nlength]) if i!= 0 and j!=0 else 1
print(np.shape(faces))

# vertices[np.asarray(left_edge)][:, 1:]

vertices *= 1/30

faces.append([0, nwidth-1, nwidth*nlength])
faces.append([nwidth-1, nwidth*nlength, nwidth*nlength+nwidth-1])

faces.append([0, nwidth*(nlength-1), nwidth*nlength])
faces.append([(nlength-1)*nwidth, nwidth*nlength, (nlength-1)*nwidth+nwidth*nlength])

faces.append([nwidth-1, nwidth-1+(nlength-1)*nwidth, nwidth-1+nwidth*nlength])
faces.append([nwidth-1+(nlength-1)*nwidth, nwidth-1+nwidth*nlength, nwidth-1+(nlength-1)*nwidth+nwidth*nlength])

faces.append([(nlength-1)*nwidth, nwidth-1+(nlength-1)*nwidth, (nlength-1)*nwidth+nwidth*nlength])
faces.append([nwidth-1+(nlength-1)*nwidth, (nlength-1)*nwidth+nwidth*nlength, nwidth-1+(nlength-1)*nwidth+nwidth*nlength])
faces = np.asarray(faces)

280 280


0it [00:00, ?it/s]

(311364, 3)


In [6]:
myramid_mesh = mesh.Mesh(
  np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype)
)
for i, f in enumerate(faces):
  for j in range(3):
      myramid_mesh.vectors[i][j] = vertices[f[j],:]

In [7]:
myramid_mesh.save('west_antarctica.stl')